In [15]:
%pip install pandas
%pip intall matplotlib.pyplot
%pip install seaborn
%pip install scikit-learn
%pip install statsmodels


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "intall" - maybe you meant "install"



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.8 MB 13.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 13.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 13.6 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.8 MB 5.0 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.8 MB 5.0 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.8 MB 5.0 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.8 MB 3.8 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.8 MB 3.8 MB/s eta 0:00:02
   ---------------------------------- ----- 8.4/9.8 MB 4.5 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.8 MB 4.5 MB/s eta 0:00:01
   ---------------------------------

In [16]:
# Importar bibliotecas
import pandas as pd # manejo de datos
import numpy as np # manejo numérico
import matplotlib.pyplot as plt # graficos
import seaborn as sns # graficos
import matplotlib.pyplot as plt


# Importar módulo para ignorar errores
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.model_selection import train_test_split # Separar el set de datos
from sklearn.linear_model import LinearRegression # Regresión Lineal
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # Importar métricas de rendimientos
import statsmodels.api as sm
pd.options.display.float_format = '{:.4f}'.format
# Importar módulo para ignorar errores
from warnings import filterwarnings
filterwarnings('ignore')

IMPORTAR SET DE DATOS

In [19]:
archivo = "ingresos-1.txt"
data = pd.read_csv(archivo, sep="\t", encoding='latin-1')

In [20]:
# Limpieza de la base de datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   estatura         1816 non-null   float64
 1   peso             1789 non-null   float64
 2   genero           1816 non-null   object 
 3   ingresos         1816 non-null   float64
 4   etnia            1816 non-null   object 
 5   educacion        1814 non-null   float64
 6   educacion_madre  1572 non-null   float64
 7   educacion_padre  1521 non-null   float64
 8   camina           1816 non-null   int64  
 9   ejercicio        1816 non-null   int64  
 10  fumador          1815 non-null   float64
 11  tenso            1815 non-null   float64
 12  malhumorado      1815 non-null   float64
 13  edad             1816 non-null   int64  
dtypes: float64(9), int64(3), object(2)
memory usage: 198.8+ KB


Vamos a ir trabajando columna a columna la base de datos.

Aprovechando que la estatura no posee valores nulos, solamente vamos a cambiar los datos para que nos muestre la información sin decimales, ya que los decimales en este caso son irrelevantes al momento de querer analizar la información, es decir, la justificación es que son imperceptibles, por lo que puede ser información extra sin sentido mantenerlos.

In [24]:
# Redondear los valores de la columna 'estatura' y actualizar la columna
data['estatura'] = data['estatura'].round(0).astype(int)

# Verificar los cambios
print(data['estatura'].head())

0    188
1    168
2    163
3    165
4    160
Name: estatura, dtype: int64


Dado que hay hartos valoes nulos para la columna peso, aprovecharemos que existen todos los valores de altura, por lo que para asignarle un valor a aquellas casillas nulas en la columna peso, nos basaremos en un IMC promedio, es decir, el promedio entre los rangos de normalidad. Buscando en internet, nos da que los rangos son entre 18.5 y 24.9, por lo que el IMC en el que nos vamos a basar es de 21.7

Entonces, todas las casillas de peso, serán rellenadas con el valor correspondiente que junto a la altura de la fila que corresponda, nos de un imc de 21.7

In [25]:
# Convertir la altura de centímetros a metros
data['estatura_metros'] = data['estatura'] / 100

# Calcular el peso basado en un IMC promedio sano de 21.7 para los valores nulos en la columna 'peso'
imc_sano = 21.7
data['peso'] = data['peso'].fillna(data['estatura_metros'] ** 2 * imc_sano)

# Verificar los cambios
print(data[['estatura', 'peso']].head())

   estatura    peso
0       188 95.2500
1       168 56.7000
2       163 57.1500
3       165 90.7200
4       160 49.9000


Comprobamos que no hay más datos nulos de peso

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   estatura         1816 non-null   int64  
 1   peso             1816 non-null   float64
 2   genero           1816 non-null   object 
 3   ingresos         1816 non-null   float64
 4   etnia            1816 non-null   object 
 5   educacion        1814 non-null   float64
 6   educacion_madre  1572 non-null   float64
 7   educacion_padre  1521 non-null   float64
 8   camina           1816 non-null   int64  
 9   ejercicio        1816 non-null   int64  
 10  fumador          1815 non-null   float64
 11  tenso            1815 non-null   float64
 12  malhumorado      1815 non-null   float64
 13  edad             1816 non-null   int64  
 14  estatura_metros  1816 non-null   float64
dtypes: float64(9), int64(4), object(2)
memory usage: 212.9+ KB


Dejamos la variable peso en solo el primer decimal, que es lo que normalmente muestra las pesas domésticas, con el mismo fin de aproximar la altura, ya que las variaciones terminan siendo de carácter imperceptible.

In [27]:
# Redondear los valores de la columna 'peso' al primer decimal
data['peso'] = data['peso'].round(1)

# Verificar los cambios
print(data[['estatura', 'peso']].head())

   estatura    peso
0       188 95.2000
1       168 56.7000
2       163 57.2000
3       165 90.7000
4       160 49.9000


Para continuar, tenemos la variable genero, y vamos a convertirla en variable binaria, así nos ayudará a cualquier intento de análisis de regresión. Cambiaremos los valores de femenino por el valor 0 y masculino por el valor 1. Luego comprobaremos que no haya ningún dato que no corresponda a 0 o 1.

In [28]:
# Convertir la columna 'genero' en una variable binaria
data['genero'] = data['genero'].replace({'femenino': 0, 'masculino': 1})

# Verificar los cambios
print(data['genero'].value_counts())

genero
0    1141
1     675
Name: count, dtype: int64


Dado que la suma entre los valores 0 y 1 es 1816, se confirma que no hay valores NA para la columna que queremos tratar como variable binaria. Además podemos observar que hay casi el doble de mujeres en esta base de datos.

Los ingresos en Dólares están bien, no hay valores nulos y están relativamente aproximados a un entero, lo que permite hacer un análisis a grandes rasgos.

Para la siguiente columna de etnia, primero verificaremos cuántos datos distintos existen.

In [29]:
# Verificar los valores únicos en la columna 'etnia'
valores_unicos = data['etnia'].unique()
cantidad_valores_unicos = data['etnia'].nunique()

# Imprimir los resultados
print("Valores únicos en 'etnia':", valores_unicos)
print("Cantidad de valores únicos en 'etnia':", cantidad_valores_unicos)

Valores únicos en 'etnia': ['Blanco' 'Other' 'Afroamericano' 'Latino']
Cantidad de valores únicos en 'etnia': 4


Dado que hay 4 etnias distintas, vamos a revisar cuántos de cada uno de estos valores únicos hay (es decir, a contarlos)

In [30]:
# Contar la cantidad de ocurrencias de cada valor en la columna 'etnia'
conteo_etnia = data['etnia'].value_counts()

# Imprimir el conteo
print(conteo_etnia)

etnia
Blanco           1494
Afroamericano     180
Latino            104
Other              38
Name: count, dtype: int64


Dado que existen en su mayoría la etnia "Blanco" la vamos a convertir en una variable dummie, para ello, la variable Blanco va a tomar el valor de 1, y todas las demás tomarán el valor 0

In [31]:
# Convertir 'etnia' en una variable dummie: 1 para 'Blanco', 0 para las demás
data['etnia'] = data['etnia'].apply(lambda x: 1 if x == 'Blanco' else 0)

# Verificar los cambios
print(data['etnia'].value_counts())

etnia
1    1494
0     322
Name: count, dtype: int64


Nuevamente se cumple con los 1816 datos no nulos.

Dado que en la columna educacion solo faltan dos datos, podemos rellenar aquellos datos faltantes con el promedio, ya que son con suerte un 0.1% de los datos.

In [32]:
# Calcular el promedio de la columna 'educacion' ignorando los valores nulos
promedio_educacion = data['educacion'].mean()

# Rellenar los valores faltantes en 'educacion' con el promedio calculado
data['educacion'] = data['educacion'].fillna(promedio_educacion)

# Verificar los cambios
print(data['educacion'].isnull().sum())  # Confirmar que no hay valores nulos

0


Además confirmamos que no hay más datos nulos.

Dado el contexto, creemos que lo mejor es dejar educación de madre y padre tal y como están, dado que pueden ser irrelevantes para nuestro estudio, y además hay demasiado valores nulos, por lo que puede ser perjudicial al análisis si es que se intenta imputar con tantos datos. Simplemente en caso de que estorben, lo mejor es ignorar estas columnas, aún no las eliminaremos. Incluso podríamos agregar los valores 0, pero indicaría que al rededor del 20% de padres y madres de muchas personas no tienen educación, lo cual podría llegar a ser un análisis relevante.

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   estatura         1816 non-null   int64  
 1   peso             1816 non-null   float64
 2   genero           1816 non-null   int64  
 3   ingresos         1816 non-null   float64
 4   etnia            1816 non-null   int64  
 5   educacion        1816 non-null   float64
 6   educacion_madre  1572 non-null   float64
 7   educacion_padre  1521 non-null   float64
 8   camina           1816 non-null   int64  
 9   ejercicio        1816 non-null   int64  
 10  fumador          1815 non-null   float64
 11  tenso            1815 non-null   float64
 12  malhumorado      1815 non-null   float64
 13  edad             1816 non-null   int64  
 14  estatura_metros  1816 non-null   float64
dtypes: float64(9), int64(6)
memory usage: 212.9 KB


Yendo ahora al tema de si caminan o no, dada la descripción de los datos al inicio, vamos a convertir la columna en sedentarismo, el salir a caminar o no, los valores de 1 a 4 se considerará como el valor binario 1, de que la persona es sedentaria. los demás valores tomarán el 0, ya qu eno sería sedentaria.

In [34]:
# Convertir la columna 'camina' en una variable binaria para sedentarismo
data['sedentario'] = data['camina'].apply(lambda x: 1 if x in [1, 2, 3, 4] else 0)

# Verificar los cambios
print(data['sedentario'].value_counts())

sedentario
0    1118
1     698
Name: count, dtype: int64


Dados los valores obtenidos, al rededor de 1/3 de la gente solamente es sedentaria.

Luego con la olumna ejercicio, se dejará de esa forma ya que es útil tener la información de días que usan las personas para hacer ejercicio a lo largo de la semana, y aquellos valores que sean 0 también tienen sentido. No hay valores nulos.

Cambiaremos los valores de la columna de si fuma o no, ya que no sirve para el análisis que los valores sean 1 y 2, mas si 0 y 1, por lo que aquellos valores iguales a 1, significa que efectivamente fuman, y en vez del valor 2, ahora se tendrá el 0 para indicar que no son fumadores.

In [35]:
# Cambiar los valores de la columna 'fumador'
data['fumador'] = data['fumador'].apply(lambda x: 1 if x == 1 else 0)

# Verificar los cambios
print(data['fumador'].value_counts())

fumador
0    1354
1     462
Name: count, dtype: int64


Sin datos nulos, ya tenemos la información en formato de variable dummie sobre si la persona es fumadora o no

Luego con si la persona está tensa o malhumorada también es información útil, y el formato al igual que en ejercicio nos puede llegar a servir para los análisis en cuestión. Aunque hay un dato nulo en ambos casos, el que rellenaremos con 0, considerando que la persona que no contetó esa información no estaba ni tensa ni malhumorada.

In [36]:
# Rellenar valores nulos en las columnas 'tenso' y 'malhumorado' con 0
data['tenso'] = data['tenso'].fillna(0)
data['malhumorado'] = data['malhumorado'].fillna(0)

# Verificar que ya no haya valores nulos
print(data[['tenso', 'malhumorado']].isnull().sum())

tenso          0
malhumorado    0
dtype: int64


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   estatura         1816 non-null   int64  
 1   peso             1816 non-null   float64
 2   genero           1816 non-null   int64  
 3   ingresos         1816 non-null   float64
 4   etnia            1816 non-null   int64  
 5   educacion        1816 non-null   float64
 6   educacion_madre  1572 non-null   float64
 7   educacion_padre  1521 non-null   float64
 8   camina           1816 non-null   int64  
 9   ejercicio        1816 non-null   int64  
 10  fumador          1816 non-null   int64  
 11  tenso            1816 non-null   float64
 12  malhumorado      1816 non-null   float64
 13  edad             1816 non-null   int64  
 14  estatura_metros  1816 non-null   float64
 15  sedentario       1816 non-null   int64  
dtypes: float64(8), int64(8)
memory usage: 227.1 KB


Revisando la información de la base de datos podemos finalmente eliminar aquellas columnas que están "duplicadas" dado todas las modificaciones que se hicieron, en este caso, eliminaremos "estatura_metros" y "camina"

In [38]:
# Eliminar las columnas 'estatura_metros' y 'camina'
data = data.drop(columns=['estatura_metros', 'camina'])

# Verificar que las columnas hayan sido eliminadas
print(data.columns)

Index(['estatura', 'peso', 'genero', 'ingresos', 'etnia', 'educacion',
       'educacion_madre', 'educacion_padre', 'ejercicio', 'fumador', 'tenso',
       'malhumorado', 'edad', 'sedentario'],
      dtype='object')


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   estatura         1816 non-null   int64  
 1   peso             1816 non-null   float64
 2   genero           1816 non-null   int64  
 3   ingresos         1816 non-null   float64
 4   etnia            1816 non-null   int64  
 5   educacion        1816 non-null   float64
 6   educacion_madre  1572 non-null   float64
 7   educacion_padre  1521 non-null   float64
 8   ejercicio        1816 non-null   int64  
 9   fumador          1816 non-null   int64  
 10  tenso            1816 non-null   float64
 11  malhumorado      1816 non-null   float64
 12  edad             1816 non-null   int64  
 13  sedentario       1816 non-null   int64  
dtypes: float64(7), int64(7)
memory usage: 198.8 KB


Y ahora con toda la base de datos limpia, es posible empezar con los distintos modelos de egresión, y verificar cuál es el mejor.

Dado que las columnas de Educación padre y madre, tienen cada uno aproximadamente 300 datos menos, vamos a eliminar estas columnas, por distintas razones.

1: La educación del padre o madre de una persona dentro de todo, a grandes rasgos debe tener una correlación baja, por lo que estamos dispuestos a un error mínimamente más grande, con el fin de no tener estos datos estorbando.

2: Es mejor eliminar las columnas que las filas que no posean esta información en particular, ya que además si no coinciden los datos faltantes de educación padre con educación madre, estamos arriesgando el eliminar en el peor de los casos casi 600 filas, y no tiene sentido eliminar 1/3 de la información.

3: Dado un largo período de reflexión, consideramos que estas columnas no serían tomadas en cuenta al momento de intentar con cualquier regresión que tengamos disponible.

4: Dejamos una base de datos completa sin datos faltantes, logrando así que toda la información disponible pueda ser utilizada sin problema.

In [40]:
#eliminar columnas
data = data.drop(columns=['educacion_madre', 'educacion_padre','estatura','ejercicio', 'fumador', 'peso'])

In [41]:
data.head()

,genero,ingresos,etnia,educacion,tenso,malhumorado,edad,sedentario
0,1,50000.0000,1,16.0000,0.0000,0.0000,45,1
1,0,60000.0000,1,16.0000,0.0000,0.0000,58,0
2,0,30000.0000,1,16.0000,1.0000,1.0000,29,0
3,0,25000.0000,1,17.0000,0.0000,0.0000,57,0
4,0,50000.0000,0,16.0000,0.0000,0.0000,91,0


Regresiones

In [92]:
def fn_metricas_info(modelo, y_verdadero, y_calculado):
    metrics = {
        'MAE': mean_absolute_error(y_verdadero, y_calculado),
        'MSE': mean_squared_error(y_verdadero, y_calculado),
        'RMSE': np.sqrt(mean_squared_error(y_verdadero, y_calculado)),
        'R2': r2_score(y_verdadero, y_calculado)
    }
    return pd.DataFrame(metrics, index=[modelo])

Regresión KKN

El método KNN es un algoritmo supervisado utilizado para predicción (regresión o clasificación) basado en encontrar los K puntos más cercanos a un dato y calcular una media o seleccionar la clase mayoritaria.

Este nos sirve para predecir los ingresos al tener un conjunto de datos amplios y con una base limpia con variables relevantes. Este se enfocara en la similitud de estos datos y generara predicciones precisas.

Instanciar el modelo Regresión KNN

In [93]:
from sklearn.neighbors import KNeighborsRegressor

y = data['ingresos']
X = data.drop(columns = 'ingresos')

# Particionar data
X_train,X_test,y_train,y_test = train_test_split(X, y, train_size = 0.8, random_state=0)


# Instanciar KNN con K=12
KNN_reg = KNeighborsRegressor(n_neighbors=12, p=1, weights='distance')
KNN_reg.fit(X_train, y_train)

y_train_pred = KNN_reg.predict(X_train)
y_test_pred = KNN_reg.predict(X_test)

pd.concat([y_train.reset_index(drop = True), pd.DataFrame(y_train_pred)], axis = 1)

,ingresos,0
0,136500.0000,71515.0000
1,28000.0000,28000.0000
2,16000.0000,16000.0000
3,50000.0000,35000.0000
4,136500.0000,136500.0000
...,...,...
1447,18000.0000,18000.0000
1448,33000.0000,33000.0000
1449,0.0000,0.0000
1450,26000.0000,26000.0000


In [94]:
fn_metricas_info('Regresión KNN',y_train, y_train_pred)

,MAE,MSE,RMSE,R2
Regresión KNN,1800.4435,34023530.7456,5832.9693,0.9347


In [95]:
fn_metricas_info('Regresión KNN',y_test, y_test_pred)

,MAE,MSE,RMSE,R2
Regresión KNN,13600.9387,397896919.0905,19947.3537,0.1216


Ajustar parámetros del modelo

In [96]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

#KNN_reg = KNeighborsRegressor(n_neighbors=12, p=1, weights='distance')

# Agregar hiperparámetros al modelo
hyperparameters = {'n_neighbors': list(range(15)),
                   'weights': ['uniform', 'distance'],
                   'p': [1, 2]}

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(knn, hyperparameters, scoring='neg_mean_absolute_error')
gs.fit(X_train, y_train)

#mostrar mejores parámetros para KNN
print(gs.best_estimator_)
print(gs.best_params_)
print(gs.best_score_)

KNeighborsRegressor(n_neighbors=14, p=1)
{'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}
-12328.862419887257


In [98]:
yk = data['ingresos']
Xk = data.drop(columns = 'ingresos')
# Particionar data
Xk_train,Xk_test,yk_train,yk_test = train_test_split(Xk, yk, train_size = 0.8, random_state=0)

# Instanciar KNN con K=14

KNN_reg2 = KNeighborsRegressor(n_neighbors=14, p=1, weights='uniform')
KNN_reg2.fit(Xk_train, yk_train)

yk_train_pred = KNN_reg2.predict(X_train)
yk_test_pred = KNN_reg2.predict(X_test)

pd.concat([yk_train.reset_index(drop = True), pd.DataFrame(yk_train_pred)], axis = 1)

,ingresos,0
0,136500.0000,34873.5714
1,28000.0000,57357.1429
2,16000.0000,18964.2857
3,50000.0000,30535.7143
4,136500.0000,31178.5714
...,...,...
1447,18000.0000,18857.1429
1448,33000.0000,42857.1429
1449,0.0000,14542.8571
1450,26000.0000,52928.5714


In [99]:
##knn ajustada
kkn= fn_metricas_info('Regresión KNN',yk_test, yk_test_pred)
kkn

,MAE,MSE,RMSE,R2
Regresión KNN,12704.8114,366964060.9990,19156.3060,0.1899


Regresion polinomial

Este enfoque se aplica cuando nuestras características sobre el target no se ajusta a una relación lineal. Un ejemplo clásico es la relación entre la caracterítica edad y salario, ya que su relación no es necesariamente lineal, sino que tiene una relación exponencial. Por lo que, es perfecto para lo que queremos predecir.

Instanciar el modelo Regresión Polinomial y aplicar a nuestro set de entrenamiento y test

In [100]:
from sklearn.preprocessing import PolynomialFeatures

yp = data['ingresos']
Xp = data.drop(columns = 'ingresos')
Xp_train,Xp_test,yp_train,yp_test = train_test_split(Xp, yp, train_size = 0.8, random_state=0)

# Instaciar modelo de regresión polinomial
poli = PolynomialFeatures(degree = 2)
Xp_poli = poli.fit_transform(X)

# Ft transformar a set de caracterìsticas
Xp_train_poli = poli.fit_transform(Xp_train)
Xp_test_poli = poli.fit_transform(Xp_test)

mod_reg_poli = LinearRegression()
# Entrenar set de prueba
mod_reg_poli.fit(Xp_train_poli, yp_train)

LinearRegression()

In [101]:
yp_train_pred = mod_reg_poli.predict(Xp_train_poli)
yp_test_pred = mod_reg_poli.predict(Xp_test_poli)

In [102]:
pd.concat([yp_train.reset_index(drop = True), pd.DataFrame(yp_train_pred)], axis = 1)

,ingresos,0
0,136500.0000,45791.0274
1,28000.0000,50337.8489
2,16000.0000,42659.8549
3,50000.0000,29362.0120
4,136500.0000,25432.5862
...,...,...
1447,18000.0000,16595.5477
1448,33000.0000,47902.8092
1449,0.0000,8041.6372
1450,26000.0000,29359.2184


In [103]:
fn_metricas_info('Regresión Polinomial G2',yp_train, yp_train_pred)

,MAE,MSE,RMSE,R2
Regresión Polinomial G2,11834.1913,384251275.0025,19602.3283,0.2625


In [104]:
polinomial= fn_metricas_info('Regresión Polinomial G2',yp_test, yp_test_pred)
polinomial

,MAE,MSE,RMSE,R2
Regresión Polinomial G2,12532.4515,358267601.5404,18927.9582,0.2091


In [105]:
dfs = [kkn, polinomial]  # Lista de DataFrames
regresiones = pd.concat(dfs, ignore_index=False)
regresiones

,MAE,MSE,RMSE,R2
Regresión KNN,12704.8114,366964060.9990,19156.3060,0.1899
Regresión Polinomial G2,12532.4515,358267601.5404,18927.9582,0.2091


Ridge

Esta regresión regularizada se utiliza para prevenir el sobreajuste del modelo. En lugar de simplemente minimizar la suma de los cuadrados de los residuos (como en la regresión lineal ordinaria), agrega una penalización a los coeficientes para controlar su magnitud, es decir, penalizar los coeficientes grandes al forzarlos a ser lo más pequeños posible. Esto nos ayudara a reducir el efecto de las variables que no sean tan relevantes para el modelo.

Determinar el mejor alfa

In [106]:
from sklearn.linear_model import Ridge

yr = data['ingresos']
Xr = data.drop(columns = 'ingresos')
Xr_train,Xr_test,yr_train,yr_test = train_test_split(Xr, yr, train_size = 0.8, random_state=0)


ridge = Ridge(alpha= 5, random_state=42)
# Entrenar modelo datos train
ridge.fit(Xr_train, yr_train)
# Obtener predicción
xr_train_pred = ridge.predict(Xr_train)
fn_metricas_info('Regresión Ridge',yr_train, xr_train_pred)

,MAE,MSE,RMSE,R2
Regresión Ridge,12178.8162,414293536.1507,20354.2019,0.2049


In [107]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold

# particiones
particiones = KFold(n_splits = 10, shuffle = True, random_state = 0)

# Definir el rango de valores de alpha
param_grid = {'alpha': [0.1, 1, 3, 5, 10]}

# Crear el modelo de regresión Ridge
ridge = Ridge()

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=particiones, scoring='neg_mean_squared_error')

# Realizar la búsqueda de cuadrícula
grid_search.fit(Xr_train, yr_train)

# Obtener el mejor valor de alpha
best_alpha = grid_search.best_params_['alpha']

print("El mejor valor de alpha es:", best_alpha)

El mejor valor de alpha es: 5


In [108]:
from sklearn.linear_model import Ridge

# Crear el modelo Ridge
ridge = Ridge()

# Ajustar el modelo a los datos de entrenamiento
ridge.fit(Xr_train, yr_train)  # Asegúrate de que Xr_train y yr_train estén definidos

# Usar el modelo para predecir en los datos de prueba
yr_test_pred = ridge.predict(Xr_test)

In [109]:
from sklearn.linear_model import Ridge

# Definir el modelo Ridge
ridge = Ridge()

# Ajustar el modelo con los datos de entrenamiento
ridge.fit(Xr_train, yr_train)  # Entrena el modelo

# Obtener predicciones en los datos de prueba
yr_test_pred = ridge.predict(Xr_test)  # Predecir los valores

# Calcular métricas (ejemplo de tu función personalizada)
Ridge = fn_metricas_info('Regresión Ridge', yr_test, yr_test_pred)

# Mostrar resultados
print(Ridge)

                       MAE            MSE       RMSE     R2
Regresión Ridge 12811.1607 365090448.5032 19107.3402 0.1940


Lasso

En la regresión Ridge los coeficientes tienden hacia cero, en la regresión Lasso los coeficientes puede volverse cero, lo que implica que la regresión Lasso tiene otro efecto y es que automáticamente depura las variables que no agregan poder predictivo al modelo. es decir este tipo de regresión implementa una selección de variables, dejando únicamente en la combinación lineal aquellas que se consideran informativas para la predicción. Tiende a generar modelos dispersos, es decir, con menos características importantes y más coeficientes exactamente iguales a cero.

La regresión Lasso es útil cuando se desea realizar una selección automática de características, ya que tiende a establecer coeficientes exactamente a cero para las características menos importantes. Sin embargo, a veces puede ser menos estable que la regresión Ridge cuando hay multicolinealidad fuerte entre las características. En tales casos, la regresión Ridge puede ser preferible.

Determinar el mejor alfa para la regresión Lasso

In [110]:
from sklearn.linear_model import Lasso
yl = data['ingresos']
Xl = data.drop(columns = 'ingresos')
Xl_train,Xl_test,yl_train,yl_test = train_test_split(Xl, yl, train_size = 0.8, random_state=0)

#ocupamos el alpha= 10
lasso = Lasso(alpha=10,random_state=42)
lasso.fit(Xl_train, yl_train)

#Obtener las predicciones del modelo
yl_train_pred = lasso.predict(Xl_train)

#@title Rendimiento del modelo
fn_metricas_info('Regresión Lasso',yl_train, yl_train_pred)

,MAE,MSE,RMSE,R2
Regresión Lasso,12182.6942,414283472.0554,20353.9547,0.2049


In [111]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold

# particiones
particiones = KFold(n_splits = 10, shuffle = True, random_state = 0)

# Definir el rango de valores de alpha
param_grid = {'alpha': [0.1, 1, 3, 5, 10]}

# Crear el modelo de regresión Ridge
lasso = Lasso()

# Configurar la búsqueda de cuadrícula
# minimizar el error cuadrático medio durante la búsqueda de cuadrícula.
grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=particiones, scoring='neg_mean_squared_error')

# Realizar la búsqueda de cuadrícula
grid_search.fit(Xl_train, yl_train)

# Obtener el mejor valor de alpha
best_alpha = grid_search.best_params_['alpha']

print("El mejor valor de alpha es:", best_alpha)

El mejor valor de alpha es: 10


In [112]:
from sklearn.linear_model import Lasso

# Crear el modelo Lasso
lasso = Lasso()

# Ajustar el modelo con los datos de entrenamiento
lasso.fit(X_train, y_train)  # Entrena el modelo con los datos de entrenamiento

# Usar el modelo para predecir los datos de prueba
y_test_pred = lasso.predict(X_test)


In [113]:
from sklearn.linear_model import Lasso

# Crear el modelo Lasso
lasso = Lasso()

# Entrenar el modelo Lasso con los datos de entrenamiento
lasso.fit(X_train, y_train)  # Asegúrate de que X_train y y_train estén definidos

# Predecir los valores con los datos de prueba
y_test_pred = lasso.predict(X_test)

# Calcular métricas usando la función personalizada
lasso_metrics = fn_metricas_info('Regresión Lasso', y_test, y_test_pred)

# Mostrar resultados
print(lasso_metrics)


                       MAE            MSE       RMSE     R2
Regresión Lasso 12812.3858 365112414.4891 19107.9150 0.1940


Determinar variables de importancia

In [114]:
modelo_final = grid_search.best_estimator_
print(f'Mejor penalización: {modelo_final}')
coeficientes = modelo_final.coef_
variables = X.columns
pd.DataFrame({'variables': variables, 'coeficientes': coeficientes})

Mejor penalización: Lasso(alpha=10)


,variables,coeficientes
0,genero,14357.9389
1,etnia,3377.9120
2,educacion,2838.9696
3,tenso,524.7712
4,malhumorado,0.0000
5,edad,193.7572
6,sedentario,-100.3690


Las variables malhumorado y sendentario son practicamente cero, por lo tanto no aportan al modelo de prediccion.

Elastic net

La principal ventaja de Elastic Net es que puede manejar conjuntos de datos con características altamente correlacionadas de manera más efectiva que Ridge o Lasso por sí solos.

In [115]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

#buscar el mejor alpha
# Definir el rango de valores de alpha
param_grid = {'alpha': [0.1, 1, 10, 100]}

# Crear el modelo de regresión Ridge
elastic_net = ElasticNet()

# particiones
particiones = KFold(n_splits = 10, shuffle = True, random_state = 0)

# Configurar la búsqueda de cuadrícula
# minimizar el error cuadrático medio durante la búsqueda de cuadrícula.
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=particiones, scoring='neg_mean_squared_error')

# Realizar la búsqueda de cuadrícula
grid_search.fit(X_train, y_train)

# Obtener el mejor valor de alpha
best_alpha = grid_search.best_params_['alpha']

print("El mejor valor de alpha es:", best_alpha)

El mejor valor de alpha es: 0.1


In [116]:
from sklearn.linear_model import ElasticNet
ye = data['ingresos']
Xe = data.drop(columns = 'ingresos')
Xe_train,Xe_test,ye_train,ye_test = train_test_split(Xe, ye, train_size = 0.8, random_state=0)

elastic_net = ElasticNet(alpha=best_alpha,
                         random_state=42)


elastic_net.fit(Xe_train, ye_train)
ye_train_pred = elastic_net.predict(X_train)
ye_test_pred = elastic_net.predict(X_test)

fn_metricas_info('Regresión Elastic Net',ye_train, ye_train_pred)

,MAE,MSE,RMSE,R2
Regresión Elastic Net,12147.2422,415955691.3648,20394.9918,0.2017


In [76]:
Elastic= fn_metricas_info('Regresión Elastic Net',ye_test, ye_test_pred)
Elastic

,MAE,MSE,RMSE,R2
Regresión Elastic Net,12769.0901,365442448.0759,19116.5491,0.1932


Como podemos notar, al comparar todas las regresiones, la Regresión Polinomial G2 es la que mejor explica el modelo. Esto se debe a que el valor de MAE es el más bajo, lo cual nos indica que, en promedio, es el modelo que menos se desvía del valor real. Con RMSE sucede algo similar. En cuanto al MSE, al penalizar los grandes errores, el más valor mas bajo nos indica ser el mejor.Por lo tanto, al ser el modelo polinomial el que cumple con todas estas características, podemos concluir que es el mejor modelo para predecir los ingresos.